### <Center> Лабораторна робота №7. <br> Алгоритм випадкового лісу в задачі кредитного скорінгу

Необхідно розв'язати задачу кредитного скорінга 

Ознаки клієнта банку:
- Age - вік (дійсночислова)
- Income - місячний дохід (дійсночислова)
- BalanceToCreditLimit - відношення балансу на кредитній картці до ліміту за кредитом (дійсночислова)
- DIR - Debt-to-income Ratio (дійсночислова)
- NumLoans - кылькість позичок і кредитних ліній
- NumRealEstateLoans - кількість іпотек і позичок, пов'язаних з нерухомістю (натуральне число)
- NumDependents - кількість членів сім'ї, яких утримує клієнт, без врахування самого клієнту (натуральне число)
- Num30-59Delinquencies - кількість протермінувань виплат за кредитом від 30 до 59 днів (натуральне число)
- Num60-89Delinquencies - кількість протермінувань виплат за кредитом від 60 до 89 дній (натуральне число)
- Delinquent90 - чи були протермінування виплат за кредитом більше 90 днів (бінарний)

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
%matplotlib inline

In [ ]:
"""
#Uncomment next sections if use in google collab
!pip install google-colab
!pip install pydrive
"""


In [3]:
"""
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
"""


In [4]:
"""
downloaded = drive.CreateFile({'id':"18dGpP5iaMmtjqnypqK5g1GpZ4cpct-9k"})
downloaded.GetContentFile('credit_scoring_train.csv')

downloaded = drive.CreateFile({'id':"1MPONmg-TxehkmcpfaERpWmfslQCqzGPH"})
downloaded.GetContentFile('credit_scoring_test.csv')
"""


Завантажимо дані з використанням pandas

In [5]:
train_df = pd.read_csv('credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('credit_scoring_test.csv', index_col='client_id')

In [6]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [7]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350


**Переглянемо кількість пропусків в кожній ознаці.**

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 0 to 74999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          73084 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 60153 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 75000 to 149999
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DIR                    75000 non-null  float64
 1   Age                    75000 non-null  float64
 2   NumLoans               75000 non-null  int64  
 3   NumRealEstateLoans     75000 non-null  int64  
 4   NumDependents          72992 non-null  float64
 5   Num30-59Delinquencies  75000 non-null  int64  
 6   Num60-89Delinquencies  75000 non-null  int64  
 7   Income                 60116 non-null  float64
 8   BalanceToCreditLimit   75000 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


**Замінимо пропуски медіанними значеннями.**

In [10]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

### Дерево рішень без налаштування параметрів

**Обучите дерево решений максимальной глубины 3, используйте параметр random_state=17 для воспроизводимости результатов.**

In [11]:
first_tree = DecisionTreeClassifier(max_depth=3, random_state=17)
first_tree.fit(train_df, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=17, splitter='best')

**Зробіть прогноз для тестової вибірки.**

In [12]:
first_tree_pred = first_tree.predict(test_df)

**Запишемо прогноз у файл.**

In [13]:
def write_to_submission_file(predicted_labels, out_file,
                             target='Delinquent90', index_label="client_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(75000, 
                                                  predicted_labels.shape[0] + 75000),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [14]:
write_to_submission_file(first_tree_pred, 'credit_scoring_first_tree.csv')

**Якщо прогнозувати ймовірності дефолту для клієнтів тестової вибірки, результат буде набагато кращим.**

In [15]:
first_tree_pred_probs = first_tree.predict_proba(test_df)[:, 1]

In [16]:
write_to_submission_file(first_tree_pred_probs, "default_probabilities.csv")

## Дерево рішень без налаштування параметрів за допомогою GridSearch

**Налаштуйте параметри дерева за допомогою `GridSearhCV`, подивіться на кращу комбінацію параметрів і середню якість на 5-кратній крос-валідації. Використовуйте параметр `random_state=17` (для відтворюваності результатів), не забувайте про розпаралелювання (`n_jobs=-1`).**

In [17]:
tree_params = {'max_depth': list(range(3, 8)), 
               'min_samples_leaf': list(range(5, 13))}
tree_model = DecisionTreeClassifier(random_state=17)
locally_best_tree = GridSearchCV(tree_model, tree_params, n_jobs=-1)
locally_best_tree.fit(train_df, y)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=17,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                        

In [18]:
locally_best_tree.best_params_, round(locally_best_tree.best_score_, 3)

({'max_depth': 5, 'min_samples_leaf': 11}, 0.935)

**Зробіть прогноз для тестової вибірки.**

In [19]:
tuned_tree_prediction = locally_best_tree.predict(test_df)

In [21]:
write_to_submission_file(tuned_tree_prediction, "tuned_tree_predict.csv")

### Випадковий ліс без настройки параметрів

**Навчіть випадковий ліс з дерев необмеженої глибини, використовуйте параметр `random_state=17` для відтворюваності результатів.**

In [22]:
first_forest = RandomForestClassifier(random_state=17)
first_forest.fit(train_df, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

In [23]:
first_forest_pred = first_forest.predict(train_df)

**Зробіть прогноз для тестової вибірки.**

In [24]:
write_to_submission_file(first_forest_pred, "first_forest_prediction.csv")

### Випадковий ліс з налаштуванням параметрів

**Налаштуйте параметр `max_features` лісу за допомогою `GridSearhCV`, подивіться на кращу комбінацію параметрів і середню якість на 5-кратній крос-валідації. Використовуйте параметр random_state=17 (для відтворюваності результатів), не забувайте про розпаралелювання (n_jobs=-1).**

In [25]:
%%time
forest_params = {'max_features': np.linspace(.3, 1, 7)}
forest_model = RandomForestClassifier(random_state=17)
locally_best_forest = GridSearchCV(forest_model, forest_params, n_jobs=-1)
locally_best_forest.fit(train_df, y)

CPU times: user 12.6 s, sys: 165 ms, total: 12.8 s
Wall time: 9min 43s


In [26]:
locally_best_forest.best_params_, round(locally_best_forest.best_score_, 3)

({'max_features': 0.3}, 0.934)

In [28]:
tuned_forest_pred = locally_best_forest.predict(test_df)

In [29]:
write_to_submission_file(tuned_forest_pred, "tuned_forest_prediction.csv")

**Подивіться, як налаштований випадковий ліс оцінює важливість ознак за їх впливом на цільову ознаку. Подайте результати в наглядному вигляді за допомогою `DataFrame`.**

In [38]:
pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_, index=train_df.columns, columns=["Importance"]).sort_values(by="Importance", ascending=False)

,Importance
BalanceToCreditLimit,0.225384
DIR,0.170093
Age,0.160560
Income,0.150816
NumLoans,0.093227
Num60-89Delinquencies,0.065174
Num30-59Delinquencies,0.063074
NumDependents,0.038709
NumRealEstateLoans,0.032962


**Як правило збільшення кількості дерев тільки покращує результат. Так що на останок навчіть випадковий ліс з 300 дерев зі знайденими кращими параметрами. Це може зайняти декілька хвилин.**

In [40]:
%%time
final_forest = RandomForestClassifier(n_estimators=300, random_state=17, **locally_best_tree.best_params_)
final_forest.fit(train_df, y)
final_forest_pred_prob = final_forest.predict_proba(test_df)[:, 1]
write_to_submission_file(final_forest_pred_prob, 'credit_scoring_probabilities_final_forest.csv')


CPU times: user 19.4 s, sys: 167 ms, total: 19.5 s
Wall time: 19.6 s
